<h1>Лабораторна робота 8</h1>
<h3>Лобунько Д.О, Обробка та аналіз БМД, БС-81</h3>

In [1]:
import pandas as pd

diet = pd.read_csv("diet.csv")
diet

,Person,gender,Age,Height,pre.weight,Diet,weight6weeks
0,25,,41,171,60,2,60.0
1,26,,32,174,103,2,103.0
2,1,0,22,159,58,1,54.2
3,2,0,46,192,60,1,54.0
4,3,0,55,170,64,1,63.3
...,...,...,...,...,...,...,...
73,74,1,35,183,83,3,80.2
74,75,1,49,177,84,3,79.9
75,76,1,28,164,85,3,79.7
76,77,1,40,167,87,3,77.8


In [2]:
groups = pd.DataFrame(columns=['Diet A', 'Diet B', 'Diet C'])
diet_A = []
diet_B = []
diet_C = []

i = 0
while True:
    if (diet['Diet'][i] == 1 and len(diet_A) < 20):
        diet_A.append(diet['pre.weight'][i]-diet['weight6weeks'][i])
    elif (diet['Diet'][i] == 2 and len(diet_B) < 20):
        diet_B.append(diet['pre.weight'][i]-diet['weight6weeks'][i])
    elif (diet['Diet'][i] == 3 and len(diet_C) < 20):
        diet_C.append(diet['pre.weight'][i]-diet['weight6weeks'][i])
    i += 1
    if (len(diet_A) == len(diet_B) == len(diet_C)):
        break
        
groups['Diet A'] = diet_A           
groups['Diet B'] = diet_B
groups['Diet C'] = diet_C
groups.index = [i+1 for i in range(len(groups))]   # 3 групи людей, які ведуть різну дієту. В кожній групі по 20 спостережень.
groups                                             # Кількісна змінна - втрата ваги протягом 6 тижнів.

,Diet A,Diet B,Diet C
1,3.8,0.0,7.0
2,6.0,0.0,5.6
3,0.7,-2.1,3.4
4,2.9,2.0,6.8
5,2.8,1.7,7.8
6,2.0,4.3,5.4
7,2.0,7.0,6.8
8,8.5,0.6,7.2
9,1.9,2.7,7.0
10,3.1,3.6,7.3


Розрахувати та вивести у вигляді таблиці статистичні характиристики даних (середні та стандартні відхилення для кожної з груп).

In [3]:
indicators = pd.DataFrame(columns=['Diet A', 'Diet B', 'Diet C'])
mean_row = pd.Series(data={'Diet A': groups['Diet A'].mean(),
                           'Diet B': groups['Diet B'].mean(),
                           'Diet C': groups['Diet C'].mean()}, name='mean')
std_row = pd.Series(data={'Diet A': groups['Diet A'].std(),
                          'Diet B': groups['Diet B'].std(),
                          'Diet C': groups['Diet C'].std()}, name='std')
sum_row = pd.Series(data={'Diet A': groups['Diet A'].sum(),
                          'Diet B': groups['Diet B'].sum(),
                          'Diet C': groups['Diet C'].sum()}, name='sum')
indicators = indicators.append(mean_row, ignore_index=False)
indicators = indicators.append(std_row, ignore_index=False)
indicators = indicators.append(sum_row, ignore_index=False)
indicators

,Diet A,Diet B,Diet C
mean,3.160000,2.620000,4.92000
std,2.420831,2.239267,2.42652
sum,63.200000,52.400000,98.40000


Побудувати графік розкиду в групах.

In [4]:
import seaborn as sns

sns.boxplot(data=groups, orient='h').set(xlabel='Loss of weight')

[Text(0.5, 0, 'Loss of weight')]

Побудувати функцію, що реалізовує однофакторний дисперсійний аналіз.

In [5]:
def ANOVA(groups):
    keys = groups.keys()
    c = len(groups.columns)
    Tc = [groups[keys[i]].sum() for i in range(c)]
    n = len(groups)
    N = n*c
    sq_xi_sum = 0
    for i in range(c):
        for j in range(1,n+1):
            sq_xi_sum += (groups[keys[i]][j])**2

    SS_fact = (sum(Tc[i]**2 for i in range(len(Tc)))/n) - ((sum(Tc)**2)/N)
    SS_zag = sq_xi_sum - (sum(Tc)**2)/N
    SS_vyp = SS_zag - SS_fact
    
    d_fact = c - 1
    d_zag = N - 1
    d_vyp = d_zag - d_fact
    
    MS_fact = SS_fact/d_fact
    MS_vyp = SS_vyp/d_vyp
    
    F_emp = MS_fact/MS_vyp
    
    Fisher_table = pd.read_excel("Fisher.xlsx")
    df_index1 = 0
    df_index2 = 0
    for i in range(1,Fisher_table.shape[0]-1):
        if (Fisher_table.iloc[i,0] <= d_vyp):
            df_index1 = i
    if (Fisher_table.iloc[Fisher_table.shape[0]-2,0] < d_vyp):
        df_index1 = Fisher_table.shape[0]-1
        
    for i in range(1,Fisher_table.shape[1]-1):
        if (Fisher_table.iloc[0,i] <= d_fact):
            df_index2 = i
    if (Fisher_table.iloc[0,Fisher_table.shape[1]-2] < d_fact):
        df_index2 = Fisher_table.shape[1]-1
    F_kryt = Fisher_table.iloc[df_index1, df_index2]
    
    table_cols = ['SS(факт)','SS(заг)','SS(вип)','d(факт)','d(заг)','d(вип)','MS(факт)','MS(вип)','F(емпір)','F(крит)']
    res_table = pd.DataFrame([SS_fact,SS_zag,SS_vyp,d_fact,d_zag,d_vyp,MS_fact,MS_vyp,F_emp,F_kryt],
                             columns=['Results'],index=table_cols)
    
    significant = True if abs(F_emp) < F_kryt else False
    if (significant == True):
        print('F(емпір) < F(крит)')
        print('significant')
    else:
        print('F(емпір) > F(крит)')
        print('non significant')
    
    return res_table

Вивести результати розрахунку кожного з етапів вказаних вище у вигляді таблиці.
Перевірити результат роботи функції на реальних або сгенерованих даних. Зробити висновок.

In [6]:
ANOVA(groups)

F(емпір) > F(крит)
non significant


,Results
SS(факт),57.861333
SS(заг),376.353333
SS(вип),318.492000
d(факт),2.000000
d(заг),59.000000
d(вип),57.000000
MS(факт),28.930667
MS(вип),5.587579
F(емпір),5.177675
F(крит),3.200000


В результаті однофакторного дисперсійного аналізу отримане значення F(емпіричне) ≈5.178, що виявилося більшим за відповідне табличне значення при ступениях свободи 2 та 57 - F(критичне) = 3.2, через що нульова гіпотеза про рівність середніх була відхилена, і, як результат, рівність середніх не була підтверджена. 